# Thesis - Step 1

Using pure text data to get best similarity scores.

In [1]:
import os

## Approach 1 - SentencePiece

This approach uses SentencePiece on text data with only the letters to try and find words.

In [2]:
import sentencepiece as spm

In [3]:
input_file = os.getcwd() + "/data/gutenberg_no_spaces.txt"
vocab_size = 8000
model_type = "unigram"
model_name = f"{model_type}_{vocab_size}"

In [ ]:
# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train(
    f"--input={input_file} --model_prefix={model_name} --vocab_size={vocab_size} --max_sentence_length=2532334 --train_extremely_large_corpus"
)

In [ ]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load(f"{model_name}.model")

# encode: text => id
print(sp.encode_as_pieces('test'))
print(sp.encode_as_ids('test'))

In [ ]:
# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([209, 31, 9, 375, 586]))